In [ ]:
# INSTALLATION
import pandas as pd
import numpy as np
!pip install babel
from babel.dates import format_date, get_month_names
import openpyxl
import requests
import re
import calendar
import os
import zipfile
import pytz
from io import BytesIO
from io import StringIO
from datetime import date, datetime, timezone, timedelta
!pip install eurostat
import eurostat
!pip install wbgapi
import wbgapi as wb
!pip install datawrapper
from datawrapper import Datawrapper
from google.colab import userdata
dw_secret = userdata.get('datawrapper1')
dw = Datawrapper(access_token=dw_secret)
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00


ValueError: mount failed

In [ ]:
# UPDATE Energieintensität der Primärenergie (Worldbank)

df = wb.data.DataFrame('EG.EGY.PRIM.PP.KD', ['DEU', 'EUU', 'OED'], labels=True)

# Speichern der Rohdaten
df.to_csv('/content/drive/MyDrive/Datenablage/et_pev_int_raw.csv', index=False)

df = df.melt(id_vars=['Country'], var_name='Jahr', value_name='Wert')
df['Jahr'] = df['Jahr'].str.replace('YR', '')
df = df.pivot(index='Jahr', columns='Country', values='Wert').reset_index()
df.rename(columns={'Germany': 'Deutschland', 'European Union': 'EU', 'OECD members': 'OECD'}, inplace=True)
df = df.reindex(columns=['Jahr', 'Deutschland', 'EU', 'OECD'])

# Speichern der Datentabelle
df.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_pev_int_y.csv', index=False)

# Update des Charts
CHART_ID = "7z0JR"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)

{'data': {'publicId': '7z0JR',
  'language': 'de-DE',
  'theme': 'dena',
  'id': '7z0JR',
  'type': 'd3-lines',
  'title': 'Energieintensität der Primärenergie',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:53:45.470Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/7z0JR/45/',
  'publicVersion': 45,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'Welt': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Frankreich': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''}},
    'upload-method': 'copy'},
   'describe': {'source-name': 'Worldbank',
    'source-url': 'https://data.worldbank.org/indicator/E

In [ ]:
# UPDATE Primärenergieverbrauch (AGEB)
# WICHTIG: Der Link muss regelmäßig geändert werden (!)

# URL zur Excel-Dateien
url_et_pev = "https://ag-energiebilanzen.de/wp-content/uploads/2024/07/02_PEV2Q.xlsx"

# Herunterladen der Excel-Dateien
response = requests.get(url_et_pev)
excel_data = BytesIO(response.content)

# Definieren des Bereichs, der eingelesen werden soll
rows_to_read = [10] + list(range(15, 23)) +[25]

# Einlesen des festgelegten Bereichs
et_pev_raw = pd.read_excel(url_et_pev, skiprows=lambda x: x not in rows_to_read).dropna(axis=1, how='all')
et_pev_raw = et_pev_raw.iloc[:, :3]

# Speichern der Rohdaten
et_pev_raw.to_csv('/content/drive/MyDrive/Datenablage/et_pev_raw.csv', index=False)

# Umrechnen der Jahresspalten in TWh
et_pev_q = et_pev_raw.copy()  # Kopie des DataFrames erstellen
et_pev_q.replace("-", 0, inplace=True)
et_pev_q.iloc[:, 1:3] = et_pev_q.iloc[:, 1:3] * 0.277778  # Multiplikation der 2. und 3. Spalte

# Erstellen einer Spalte, in der die prozentuale Veränderung zwischen den Jahreswerten ermittelt wird / Wegen des Stromaustauschsaldos gibt es die Vorzeichenbedingung
et_pev_q['Veränderung (%)'] = et_pev_q.iloc[:, 1:3].apply(lambda col: (col.iloc[1] - col.iloc[0]) / col.iloc[0] * 100 if col.iloc[0] * col.iloc[1] >= 0 else "NA", axis=1)

# Speichern der Datentabelle
et_pev_q.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_pev_q.csv', index=False)

# Update des Charts
CHART_ID = "onxW3"
dw.add_data(chart_id=CHART_ID, data=et_pev_q)
dw.publish_chart(chart_id=CHART_ID, display=False)

<ipython-input-3-c577c583425a>:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  et_pev_q.replace("-", 0, inplace=True)


{'data': {'publicId': 'onxW3',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'onxW3',
  'type': 'tables',
  'title': 'Primärenergieverbrauch nach Energieträgern',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:53:50.408Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/onxW3/94/',
  'publicVersion': 94,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [{'id': '3kbG0Pzj6e',
      'row': 0,
      'time': 1586249356262,
      'value': 'Country',
      'column': 0,
      'ignored': False,
      'previous': 'Country Name'},
     {'id': '025uzMTg7m',
      'row': 0,
      'time': 1701437220632,
      'value': 'Veränderu',
      'column': 3,
      'ignored': False,
      'previous': 'Veränderung (%)'},
     {'id': 'uOaheS5z3t',
      'row': 0,
      'time': 1701437223623,
      'value': 'Veränderug',
      'column': 3,
      'ignored': False,
      'previous': 'Veränderu'},
     {'id': 'lYqs_-cgYh',
      'row': 0,
  

In [ ]:
# UPDATE Nettostromerzeugung (SMARD)

# Definition der Funktion zum Einlesend der SMARD Daten
def get_json_smard(filter_num, name, freq="month", start_year=2015, end_year=None):
    if end_year is None:
        end_year = datetime.now().year
    data_frames = []

    for year in range(start_year, end_year + 1):
        tz_offset = timedelta(hours=1)  # UTC+1
        timestamp = int((datetime(year, 1, 1, tzinfo=timezone(tz_offset)) - datetime(1970, 1, 1, tzinfo=timezone.utc)).total_seconds()) * 1000
        link = f"https://www.smard.de/app/chart_data/{filter_num}/DE/{filter_num}_DE_{freq}_{timestamp}.json"

        response = requests.get(link)
        if response.status_code == 200:
            data = pd.DataFrame(response.json().get('series', []), columns=['V1', 'V2'])
            if not data.empty:
                data['Datum'] = pd.to_datetime(data['V1'], unit='ms', utc=True).dt.tz_convert('Europe/Berlin').dt.date
                data[name] = data['V2']
                data_frames.append(data[['Datum', name]])
        else:
            print(f"Failed to fetch data for year {year}. Status code: {response.status_code}")

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        return pd.DataFrame()

def combine_dataframes(filter_name_list, freq="month", start_year=2015, end_year=None):
    dfs = [get_json_smard(filter_num, name, freq=freq, start_year=start_year, end_year=end_year) for filter_num, name in filter_name_list]
    combined_df = pd.concat(dfs, axis=1, join='outer')
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
    return combined_df

filter_name_list = [
    (1223, "Braunkohle"), (1224, "Kernenergie"), (1225, "Wind Offshore"), (1226, "Wasserkraft"),
    (1227, "Sonstige Konventionelle"), (1228, "Sonstige Erneuerbare"), (4066, "Biomasse"),
    (4067, "Wind Onshore"), (4068, "Photovoltaik"), (4069, "Steinkohle"), (4070, "Pumpspeicher"),
    (4071, "Erdgas")
]

et_nettostrom_raw = combine_dataframes(filter_name_list)
et_nettostrom_raw.to_csv('/content/drive/MyDrive/Datenablage/et_nettostrom_raw.csv', index=False)

order_lev = ["Datum", "Erdgas", "Kernenergie", "Braunkohle", "Steinkohle", "Sonstige Konventionelle",
             "Pumpspeicher", "Sonstige Erneuerbare", "Biomasse", "Wasserkraft", "Wind Offshore", "Wind Onshore",
             "Photovoltaik"]
et_nettostrom_m = et_nettostrom_raw[order_lev].copy()
et_nettostrom_m.iloc[:, 1:] /= 1000000

et_nettostrom_m.loc[:, "Wind"] = et_nettostrom_m["Wind Offshore"] + et_nettostrom_m["Wind Onshore"]
et_nettostrom_m.loc[:, "Sonstige Konventionelle"] += et_nettostrom_m["Pumpspeicher"]
et_nettostrom_m.loc[:, "Sonstige Erneuerbare"] += et_nettostrom_m["Wasserkraft"]

et_nettostrom_m = et_nettostrom_m.drop(columns=["Wind Offshore", "Wind Onshore", "Pumpspeicher", "Wasserkraft"])

# Speichern der Datentabelle
et_nettostrom_m.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_nettostrom_m.csv', index=False)

# Update des Charts
CHART_ID = "sP7lF"
dw.add_data(chart_id=CHART_ID, data=et_nettostrom_m)
dw.publish_chart(chart_id=CHART_ID, display=False)

Failed to fetch data for year 2024. Status code: 404


{'data': {'publicId': 'sP7lF',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'sP7lF',
  'type': 'd3-area',
  'title': 'Nettostromerzeugung',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:54:40.306Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/sP7lF/108/',
  'publicVersion': 108,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'X.1': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Wind': {'type': 'auto',
      'ignore': False,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Monat': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-p

In [ ]:
# UPDATE Nettostromerzeugung (SMARD) - Laufendes Jahr
# Basis ist der zuvor erzeugte Dataframe et_nettostrom_m

# DataFrame transponieren und "Datum" als Index setzen
et_nettostrom_m_ytd = et_nettostrom_m.set_index('Datum').transpose().reset_index().rename_axis(columns=None)

# Automatisierung, um nur das laufende Jahr anzuzeigen
# Höchstes Jahr ermitteln
max_jahr = max(pd.to_datetime(et_nettostrom_m_ytd.columns[1:], errors='coerce', format='%Y-%m-%d').year)

# Spalten mit dem höchsten Jahr auswählen
spalten_hoechstes_jahr = [col for col in et_nettostrom_m_ytd.columns[1:] if pd.to_datetime(col, errors='coerce', format='%Y-%m-%d').year == max_jahr]

# DataFrame mit Spalten des höchsten Jahres erstellen
et_nettostrom_m_ytd = et_nettostrom_m_ytd[['index'] + spalten_hoechstes_jahr]
et_nettostrom_m_ytd = et_nettostrom_m_ytd.rename(columns={'index': 'Energieträger'})

# Monatsnamen auf Deutsch
month_names_de = {
    1: 'Januar',
    2: 'Februar',
    3: 'März',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Dezember'
}

# Spaltennamen umwandeln
new_columns = []
for col in et_nettostrom_m_ytd.columns:
    if col != 'Energieträger':
        month_name_de = month_names_de[col.month]
        new_columns.append(month_name_de)
    else:
        new_columns.append(col)


# Umbenennung der Spalten im DataFrame
et_nettostrom_m_ytd.columns = new_columns

et_nettostrom_m_ytd = et_nettostrom_m_ytd.fillna('na')

# Speichern der Datentabelle
et_nettostrom_m_ytd.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_nettostrom_m_ytd.csv', index=False)

# Update des Charts
CHART_ID = "vb16L"
dw.add_data(chart_id=CHART_ID, data=et_nettostrom_m_ytd)
dw.publish_chart(chart_id=CHART_ID, display=False)


{'data': {'publicId': 'vb16L',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'vb16L',
  'type': 'stacked-column-chart',
  'title': 'Nettostromerzeugung',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:54:43.799Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/vb16L/98/',
  'publicVersion': 98,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'X.1': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Monat': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Summe': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
    

In [ ]:
# UPDATE Nettostromerzeugung (SMARD) - Anteil Erneuerbare
# Basis ist der zuvor erzeugte Dataframe et_nettostrom_m


et_ee_nettostrom_m = et_nettostrom_m.copy()
et_ee_nettostrom_m = et_ee_nettostrom_m.fillna(0)

# Erzeugen einer "Insgesamt"-Spalte
et_ee_nettostrom_m['Insgesamt'] = et_ee_nettostrom_m.iloc[:, 1:].sum(axis=1, skipna=False)

# Liste der Spalten, die für die "Erneuerbare" und "Konventionelle"-Spalten berücksichtigt werden sollen
erneuerbare_spalten = ['Biomasse', 'Photovoltaik', 'Sonstige Erneuerbare', 'Wind']
konventionelle_spalten = ['Erdgas', 'Kernenergie', 'Braunkohle', 'Steinkohle', 'Sonstige Konventionelle']

# Hinzufügen der Spalten
et_ee_nettostrom_m['Erneuerbare'] = et_ee_nettostrom_m[erneuerbare_spalten].sum(axis=1, skipna=False)
et_ee_nettostrom_m['Konventionelle'] = et_ee_nettostrom_m[konventionelle_spalten].sum(axis=1, skipna=False)

# Konvertiere jede Spalte zu numerischen Werten
for col in erneuerbare_spalten + ['Konventionelle', 'Insgesamt']:
    et_ee_nettostrom_m[col] = pd.to_numeric(et_ee_nettostrom_m[col], errors='coerce')

# Füge die Spalten "Anteil Erneuerbare" und "Anteil Konventionelle" hinzu
et_ee_nettostrom_m['Anteil Erneuerbare'] = et_ee_nettostrom_m['Erneuerbare'] / et_ee_nettostrom_m['Insgesamt']*100
et_ee_nettostrom_m['Anteil Konventionelle'] = et_ee_nettostrom_m['Konventionelle'] / et_ee_nettostrom_m['Insgesamt']*100

# Ersetze NaN-Werte in den Anteils-Spalten durch "NA"
et_ee_nettostrom_m[['Anteil Erneuerbare', 'Anteil Konventionelle']] = et_ee_nettostrom_m[['Anteil Erneuerbare', 'Anteil Konventionelle']].replace(np.nan, 'NA')

# Ausgabe des resultierenden DataFrames
et_ee_nettostrom_m = et_ee_nettostrom_m[['Datum', 'Anteil Erneuerbare', 'Anteil Konventionelle']]

# Speichern der Datentabelle
et_ee_nettostrom_m.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_ee_nettostrom_m.csv', index=False)

# Update des Charts
CHART_ID = "0tfVF"
dw.add_data(chart_id=CHART_ID, data=et_ee_nettostrom_m)
dw.publish_chart(chart_id=CHART_ID, display=False)

{'data': {'publicId': '0tfVF',
  'language': 'de-DE',
  'theme': 'dena',
  'id': '0tfVF',
  'type': 'd3-area',
  'title': 'Nettostromerzeugung',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:54:47.694Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/0tfVF/107/',
  'publicVersion': 107,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [{'id': 'XGGNKpLsD1',
      'row': 0,
      'time': 1701688280467,
      'value': 'Erneuerbare',
      'column': 1,
      'ignored': False,
      'previous': 'Anteil Erneuerbare'},
     {'id': '5E2rykyOJ3',
      'row': 0,
      'time': 1701688284818,
      'value': 'Konventionelle',
      'column': 2,
      'ignored': False,
      'previous': 'Anteil Konventionelle'}],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'X.1': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      '

In [ ]:
#UPDATE Erdgasverbrauch (BNetzA)

# URL zur Excel-Dateien
url_gasverb = "https://www.bundesnetzagentur.de/_tools/SVG/js2/_functions/csv_export.html?view=renderCSV&id=1093162"

# Herunterladen der Excel-Dateien
response = requests.get(url_gasverb)
excel_data = BytesIO(response.content)

# Einlesen der Daten
et_gasverb_raw = pd.read_csv(url_gasverb, delimiter=';')

# Speichern der Rohdaten
et_gasverb_raw.to_csv('/content/drive/MyDrive/Datenablage/et_gasverb_raw.csv', index=False)

# Umbenennen der Datumsspalte
et_gasverb_raw = et_gasverb_raw.rename(columns={'                     .': 'Monat'})


# Bestimme den Namen für die neue Spalte
name_urspruengliche_spalte = et_gasverb_raw.columns[3]
jahr = int(name_urspruengliche_spalte.split()[-1]) + 1

# Erzeuge den neuen Spaltennamen
neuer_spaltenname = f"{name_urspruengliche_spalte.split()[0]} {jahr}"


# Füge die neue Spalte mit dem dynamischen Namen hinzu und fülle sie mit den Werten der 1. und 2. Spalte
et_gasverb_raw[neuer_spaltenname] = et_gasverb_raw.iloc[:, [1, 2]].sum(axis=1, skipna=False)

et_gasverb_m = et_gasverb_raw.iloc[:, [0, 1, 2, 6, 3, 4]]
et_gasverb_m = et_gasverb_m.fillna('NA')

et_gasverb_m['Monat'] = et_gasverb_m['Monat'].apply(lambda x: format_date(date(2022, x, 1), "MMMM", locale='de_DE'))


# Speichern der Datentabelle
et_gasverb_m.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_gasverb_m.csv', index=False)


# Update des Charts
CHART_ID = "AfeT8"
dw.add_data(chart_id=CHART_ID, data=et_gasverb_m)
dw.publish_chart(chart_id=CHART_ID, display=False)


{'data': {'publicId': 'AfeT8',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'AfeT8',
  'type': 'd3-lines',
  'title': 'Erdgasverbrauch in Deutschland',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:54:52.299Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/AfeT8/94/',
  'publicVersion': 94,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'Jahr': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Monat': {'type': 'date',
      'ignore': False,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Industrie': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,

In [ ]:
# UPDATE Endenergieverbrauch nach Sektoren (AGEB)
# WICHTIG: Der Link muss regelmäßig geändert werden (!)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# URL zur Excel-Dateien
url_et_eev_sek = "https://ag-energiebilanzen.de/wp-content/uploads/2023/11/awt_2023_d.xlsx"

# Herunterladen der Excel-Dateien
response = requests.get(url_et_eev_sek, headers=headers)
excel_data = BytesIO(response.content)

rows_to_read = [3] + list(range(12, 16))

# Einlesen der Daten aus Excel-Datei
et_eev_sek_raw = pd.read_excel(excel_data, sheet_name="2.2", engine='openpyxl', skiprows=lambda x: x not in rows_to_read).dropna(axis=1, how='all')

# Speichern der Rohdaten
et_eev_sek_raw.to_csv('/content/drive/MyDrive/Datenablage/et_eev_raw.csv', index=False)

# Auswahl relevanter Spalten
et_eev_sek_j = et_eev_sek_raw.drop(columns=['Einheit'])

# Umrechnung PJ in TWh
et_eev_sek_j.iloc[:, 1:] = et_eev_sek_j.iloc[:, 1:].apply(lambda x: x * 0.277778)

# Speichern der Datentabelle
et_eev_sek_j.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_eev_sek_j.csv', index=False)

# Update des Chart
CHART_ID = "3iTH8"
dw.add_data(chart_id=CHART_ID, data=et_eev_sek_j)
dw.publish_chart(chart_id=CHART_ID, display=False)

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Datenablage'

In [ ]:
# UPDATE Endenergieverbrauch nach Energieträgern (AGEB)
# WICHTIG: Der Link muss regelmäßig geändert werden (!)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# URL zur Excel-Dateien
url_et_eev_sek = "https://ag-energiebilanzen.de/wp-content/uploads/2023/11/awt_2023_d.xlsx"

# Herunterladen der Excel-Dateien
response = requests.get(url_et_eev_sek, headers=headers)
excel_data = BytesIO(response.content)

rows_to_read = [3] + list(range(5, 14))

# Einlesen der Daten aus Excel-Datei
et_eev_raw = pd.read_excel(excel_data, sheet_name="6.1", engine='openpyxl', skiprows=lambda x: x not in rows_to_read).dropna(axis=1, how='all')

# Speichern der Rohdaten
et_eev_raw.to_csv('/content/drive/MyDrive/Datenablage/et_eev_raw.csv', index=False)

# Auswahl relevanter Spalten
et_eev_j = et_eev_raw.drop(columns=['Einheit'])

# Löschen der Zeile 'Erdgas, Erdölgas', da in 'Gase' enthalten
value_to_delete = 'Erdgas, Erdölgas'
et_eev_j.drop(et_eev_j[et_eev_j['Energieträger'] == value_to_delete].index, inplace=True)

# Umrechnung PJ in TWh
et_eev_j.iloc[:, 1:] = et_eev_j.iloc[:, 1:].apply(lambda x: x * 0.277778)

# Speichern der Datentabelle
et_eev_j.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_eev_j.csv', index=False)

# Update des Chart
CHART_ID = "FvFGs"
dw.add_data(chart_id=CHART_ID, data=et_eev_j)
dw.publish_chart(chart_id=CHART_ID, display=False)

NameError: name 'requests' is not defined

In [ ]:
# UPDATE Netzlast (SMARD)

# Definition der Funktion um Daten von SMARD-API abzurufen
def get_json_smard(filter_num, name, freq="day", start_year=2021, end_year=None):
    if end_year is None:
        end_year = datetime.now().year

    data_frames = []

    for year in range(start_year, end_year + 1):
        tz_offset = timedelta(hours=1)  # UTC+1
        timestamp = int((datetime(year, 1, 1, tzinfo=timezone(tz_offset)) - datetime(1970, 1, 1, tzinfo=timezone.utc)).total_seconds()) * 1000
        link = f"https://www.smard.de/app/chart_data/{filter_num}/DE/{filter_num}_DE_{freq}_{timestamp}.json"

        response = requests.get(link)

        if response.status_code == 200:
            data = pd.DataFrame(response.json().get('series', []), columns=['V1', 'V2'])
            if not data.empty:
                data['Datum'] = pd.to_datetime(data['V1'], unit='ms', utc=True).dt.tz_convert('Europe/Berlin').dt.date
                data[name] = data['V2']
                data = data[['Datum', name]]
                data_frames.append(data)
            else:
                print(f"No data for year {year}")
        else:
            print(f"Failed to fetch data for year {year}. Status code: {response.status_code}")

    # Daten kombinieren
    if data_frames:
        result = pd.concat(data_frames, ignore_index=True)
        return result
    else:
        print("No data available for the specified years.")
        return pd.DataFrame()

# Datenabruf
filter_num = "410"
name_def = "Stromverbrauch (Netzlast)"
et_stromverb_raw = get_json_smard(filter_num, name_def)

# Speichern der Rohdaten
et_stromverb_raw.to_csv('/content/drive/MyDrive/Datenablage/et_stromverb_raw.csv', index=False)

# Konvertieren Sie die "Datum"-Spalte in den Datetime-Typ
et_stromverb_raw['Datum'] = pd.to_datetime(et_stromverb_raw['Datum'])

# Jahr extrahieren und DataFrame nach Jahren trennen
et_stromverb_raw['Jahr'] = et_stromverb_raw['Datum'].dt.year
df_by_year = et_stromverb_raw.pivot(index='Datum', columns='Jahr', values='Stromverbrauch (Netzlast)')

# Aktuelles Jahr extrahieren
current_year = pd.Timestamp.now().year

# Aktuelle Kalenderwoche extrahieren
current_week = pd.Timestamp.now().isocalendar()[1]

# Wochenmittelwerte pro Jahr berechnen
et_stromverb_w = df_by_year.groupby(df_by_year.index.isocalendar().week).mean()

# Filtern der Wochen für das aktuelle Jahr Füllen der fehlenden Werte mit "NA"
et_stromverb_w[current_year] = et_stromverb_w[current_year].where(et_stromverb_w.index <= current_week - 1, 'NA')

# Teilen Sie nur numerische Werte im DataFrame durch 1.000.000
et_stromverb_w = et_stromverb_w.applymap(lambda x: x / 1000000 if pd.notna(x) and isinstance(x, (int, float)) else x)

# Umbenennen der Spalten
et_stromverb_w.insert(0, 'Durchschnittliche Netzlast pro KW', et_stromverb_w.index)
et_stromverb_w.columns = [str(col) for col in et_stromverb_w.columns]

# Hinzufügen des Zusatzes "KW"
et_stromverb_w['Durchschnittliche Netzlast pro KW'] = 'KW ' + et_stromverb_w['Durchschnittliche Netzlast pro KW'].astype(str)

# Speichern der Datentabelle
et_stromverb_w.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_stromverb_w.csv', index=False)

# Update des Chart
CHART_ID = "0jN5z"
dw.add_data(chart_id=CHART_ID, data=et_stromverb_w)
dw.publish_chart(chart_id=CHART_ID, display=False)

<ipython-input-10-c34f4cb540fd>:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  et_stromverb_w = et_stromverb_w.applymap(lambda x: x / 1000000 if pd.notna(x) and isinstance(x, (int, float)) else x)


{'data': {'publicId': '0jN5z',
  'language': 'de-DE',
  'theme': 'dena',
  'id': '0jN5z',
  'type': 'd3-lines',
  'title': 'Stromverbrauch (Netzlast)',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:55:09.126Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/0jN5z/88/',
  'publicVersion': 88,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'2021': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''}},
    'external-data': 'https://raw.githubusercontent.com/anneberner/Tabellen_duke/main/et_stromverb_h',
    'upload-method': 'upload',
    'use-datawrapper-cdn': False},
   'describe': {'source-name': 'SMARD',
    'source-url': '',
    'intro': 'Wöchentliche Mittelwerte <br>\nin TWh/Tag',
    'byline': 'dena',
  

In [ ]:
# UPDATE Zubau EE (Bundesnetzagentur)

# URL zur Excel-Dateien
url_ausbau_ee = "https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/ErneuerbareEnergien/ZahlenDatenInformationen/EEStatistikMaStR_XLS.xlsx?__blob=publicationFile&v=11"

# Herunterladen der Excel-Dateien
response = requests.get(url_ausbau_ee)
excel_data = BytesIO(response.content)

# Definieren des Bereichs, der eingelesen werden soll
rows_to_read = [63] + list(range(65, 76))

# Einlesen des festgelegten Bereichs
et_ausbau_ee_raw = pd.read_excel(excel_data, skiprows=lambda x: x not in rows_to_read)

# Spalten mit "Unnamed" entfernen
et_ausbau_ee_raw = et_ausbau_ee_raw.dropna(axis=1, how="all")

# Speichern der Rohdaten
et_ausbau_ee_raw.to_csv('/content/drive/MyDrive/Datenablage/et_ausbau_ee_raw.csv', index=False)

# Umbenennen der Spalten
et_ausbau_ee_m = et_ausbau_ee_raw.rename(columns={"Unnamed: 0": "Monat", "Solare Strahlungsenergie" : "PV - Gesamt"})

print(et_ausbau_ee_m)

# Monatsspalte in Datetime-Format konvertieren
et_ausbau_ee_m["Monat"] = pd.to_datetime(et_ausbau_ee_m["Monat"], format="%Y-%m-%d")

# Speichern der Datentabelle
et_ausbau_ee_m.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_ausbau_ee_m.csv', index=False)

# Update der Charts
CHART_ID_ges = "lqsbi"
dw.add_data(chart_id=CHART_ID_ges, data=et_ausbau_ee_m)
dw.publish_chart(chart_id=CHART_ID_ges, display=False)

CHART_ID_biom = "y4g78"
dw.add_data(chart_id=CHART_ID_biom, data=et_ausbau_ee_m)
dw.publish_chart(chart_id=CHART_ID_biom, display=False)

CHART_ID_pv = "slhw4"
dw.add_data(chart_id=CHART_ID_pv, data=et_ausbau_ee_m)
dw.publish_chart(chart_id=CHART_ID_pv, display=False)

CHART_ID_onshore = "brtcQ"
dw.add_data(chart_id=CHART_ID_onshore, data=et_ausbau_ee_m)
dw.publish_chart(chart_id=CHART_ID_onshore, display=False)

CHART_ID_offshore = "QiauD"
dw.add_data(chart_id=CHART_ID_offshore, data=et_ausbau_ee_m)
dw.publish_chart(chart_id=CHART_ID_offshore, display=False)

        Monat  Biomasse  PV - Gesamt  Windenergie an Land  Windenergie auf See
0  2023-10-01   2.87790   1502.01550            411.89315                0.000
1  2023-11-01  14.23500   1419.80197            199.99240               28.575
2  2023-12-01 -22.72804   1148.72730            369.30310                0.000
3  2024-01-01   1.98640   1583.68298            130.16900                0.000
4  2024-02-01   6.60400   1223.35398            215.47630                0.000
5  2024-03-01  15.74600   1364.48221            212.19092                0.000
6  2024-04-01  19.11900   1279.98313            142.21880                0.000
7  2024-05-01  24.27800   1169.50083             44.74900              222.300
8  2024-06-01   6.64790   1367.67657            176.91340              155.025
9  2024-07-01  -1.20130   1518.61119            216.18300              189.075
10 2024-08-01  21.96600    905.94654            188.82010               87.750


{'data': {'publicId': 'QiauD',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'QiauD',
  'type': 'column-chart',
  'title': 'Erneuerbare Energieträger - Entwicklung des Netto-Zubaus der Bruttoleistung',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T09:58:43.847Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/QiauD/88/',
  'publicVersion': 88,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [{'id': 'kVQ7aM6ENe',
      'row': 0,
      'time': 1697617699160,
      'value': 'PV',
      'column': 4,
      'ignored': False,
      'previous': 'PV - Gesamt'}],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'Jahr': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Monat': {'type': 'auto',
      'ignore': False,
      'number-append': '',
      'number-for

In [ ]:
# NEU Windenergie Kennzahlen (MaStR)
# Nabehöhe
# Rotordurchmesser
# Nettoleistung
# Brutooleistung
# Inbetriebnahmen nach 01.01.2000
# Inbetriebnahme vor "aktuelles Datum"
# Energieträger entspricht Wind
#eventuell nicht mehr in Betrieb

base_url = "https://www.marktstammdatenregister.de/MaStR/Einheit/EinheitJson/GetErweiterteOeffentlicheEinheitStromerzeugung?sort=&page=1&pageSize=25000&group=&filter=Nabenh%C3%B6he%20der%20Windenergieanlage~nnn~%27%27~and~Rotordurchmesser%20der%20Windenergieanlage~nnn~%27%27~and~Nettonennleistung%20der%20Einheit~nnn~%27%27~and~Bruttoleistung%20der%20Einheit~nnn~%27%27~and~Inbetriebnahmedatum%20der%20Einheit~lt~%2731.12.{year}%27~and~Inbetriebnahmedatum%20der%20Einheit~gt~%2701.01.{year}%27~and~Energietr%C3%A4ger~eq~%272497%27"

# Aktuelles Jahr ermitteln
current_year = datetime.now().year

data_frames = []

# JSON-Zeitstempel definieren
def convert_json_date(json_date):
    # Extrahiere die Millisekunden aus dem JSON-Datum
    milliseconds = int(re.search(r'\d+', json_date).group(0))

    # Konvertiere Millisekunden in Sekunden und dann in ein Datumsobjekt
    date = datetime.utcfromtimestamp(milliseconds / 1000)

    return date

# Loop über jedes Jahr von 2000 bis zum aktuellen Jahr
for year in range(2000, current_year + 1):
    url = base_url.format(year=year)

    response = requests.get(url)
    data = response.json()
    data = data['Data']

    # Konvertiere das Datum in der Spalte 'InbetriebnahmeDatum' in das richtige Format
    for entry in data:
        entry['InbetriebnahmeDatum'] = convert_json_date(entry['InbetriebnahmeDatum'])

    df = pd.DataFrame(data, columns=['MaStRNummer', 'InbetriebnahmeDatum', 'NabenhoeheWindenergieanlage',
                                     'Nettonennleistung', 'RotordurchmesserWindenergieanlage', 'LageEinheit', 'LageEinheitBezeichnung'])
    data_frames.append(df)

# Zusammenführen der DataFrames
df = pd.concat(data_frames, ignore_index=True)

#Zeit in Monat-Jahr
df['Zeit'] = df['InbetriebnahmeDatum'].dt.strftime('%m-%Y')

# Durchschnittwert verschiedener Spalten
df = df.groupby(['Zeit','LageEinheit'])[['NabenhoeheWindenergieanlage', 'Nettonennleistung', 'RotordurchmesserWindenergieanlage']].mean()
df = df.reset_index()

# Umrechnene Nettoleistung in kW zu hW (Hektowatt)
df['Nettonennleistung'] = df['Nettonennleistung'] / 100

# Umbenennung und Änderung der Reihenfolge der Spalten
df = df.rename(columns={'NabenhoeheWindenergieanlage': 'Nabenhöhe in m', 'RotordurchmesserWindenergieanlage': 'Rotordurchmesser in m', 'Nettonennleistung': 'Leistung in hW'})

Reihenfolge = {'Zeit' : 0, 'LageEinheit': 1, 'Leistung in hW': 2, 'Nabenhöhe in m': 3, 'Rotordurchmesser in m': 4}
df = df[sorted(df.columns, key=lambda x: Reihenfolge.get(x, float('inf')))]

# Speichern in Datenablage
df.to_csv('/content/drive/MyDrive/Datenablage/et_wind_neubest_m.csv', index=False)

# Unterteilen in See und Land
df_see = df.loc[df['LageEinheit'] == 889]
df_see = df_see.drop(columns=['LageEinheit'])

df_land = df.loc[df['LageEinheit'] == 888]
df_land = df_land.drop(columns=['LageEinheit'])

# Konvertierung der Spalte 'Zeit' in ein Datumsformat
df_land['Zeit'] = pd.to_datetime(df_land['Zeit'], format='%m-%Y')
df_see['Zeit'] = pd.to_datetime(df_see['Zeit'], format='%m-%Y')

# DataFrame nach der konvertierten Spalte "Zeit" sortieren
df_land = df_land.sort_values(by='Zeit')
df_see = df_see.sort_values(by='Zeit')

# Aktuelles Datum abrufen und zum Vormonat wechseln
heute = datetime.now()
vormonat = heute - timedelta(days=heute.day)

# Filtern des DataFrames, um nur Werte bis zum Vormonat anzuzeigen
df_land = df_land[df_land['Zeit'] <= vormonat]
df_see = df_see[df_see['Zeit'] <= vormonat]

# Datumsformat einkürzen
df_land['Zeit'] = df_land['Zeit'].dt.strftime('%m-%Y')
df_see['Zeit'] = df_see['Zeit'].dt.strftime('%m-%Y')

# Speichern
df_see.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_wind_see_m.csv', index=False)
df_land.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_wind_land_m.csv', index=False)

# Veröffentlichung von Anlagen an Land
CHART_ID = "ophlH"
dw.add_data(chart_id=CHART_ID, data=df_land)
dw.publish_chart(chart_id=CHART_ID, display=False)

{'data': {'publicId': 'ophlH',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'ophlH',
  'type': 'd3-lines',
  'title': 'Neuinbetriebnahme von Windenergie an Land: Technische Konfiguration und Leistung',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T10:00:08.700Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/ophlH/52/',
  'publicVersion': 52,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {'LageEinheit': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Bruttoleistung': {'type': 'auto',
      'ignore': True,
      'number-append': '',
      'number-format': 'auto',
      'number-divisor': 0,
      'number-prepend': ''},
     'Nettonennleistung': {'type': 'auto',
      'ignore': False,
      'num

In [ ]:
# NEU Windräder Anzahl (MaStR)
#Energieträger entspricht Wind
#Inbetriebnahme nach 31.12.2023
#Postleitzahl ist nicht leer
#Breitengrad ist nicht leer
#Längengrad ist nicht leer

def convert_json_date(json_date):
    # Extrahiere die Millisekunden aus dem JSON-Datum
    milliseconds = int(re.search(r'\d+', json_date).group(0))

    # Konvertiere Millisekunden in Sekunden und dann in ein Datumsobjekt
    date = datetime.utcfromtimestamp(milliseconds / 1000)

    return date

# Aktuelles Jahr bekommen und um eins reduzieren
current_year = datetime.now().year - 1

# Link mit dem aktuellen Jahr füllen
#url = f"https://www.marktstammdatenregister.de/MaStR/Einheit/EinheitJson/GetErweiterteOeffentlicheEinheitStromerzeugung?sort=&page=1&pageSize=25000&group=&filter=Energietr%C3%A4ger~eq~%272497%27~and~Inbetriebnahmedatum%20der%20Einheit~gt~%2731.12.{current_year}%27"
url = f"https://www.marktstammdatenregister.de/MaStR/Einheit/EinheitJson/GetErweiterteOeffentlicheEinheitStromerzeugung?sort=&page=1&pageSize=25000&group=&filter=Koordinate%3A%20L%C3%A4ngengrad%20(WGS84)~nnn~%27%27~and~Koordinate%3A%20Breitengrad%20(WGS84)~nnn~%27%27~and~Postleitzahl~nn~%27%27~and~Energietr%C3%A4ger~eq~%272497%27~and~Inbetriebnahmedatum%20der%20Einheit~gt~%2731.12.{current_year}%27"

# Anfrage senden
response = requests.get(url)
data = response.json()
data = data['Data']

# Daten in DataFrame laden
df = pd.DataFrame(data, columns=['Plz', 'Breitengrad', 'Laengengrad', 'LageEinheit', 'LageEinheitBezeichnung', 'InbetriebnahmeDatum','Bruttoleistung'])

# Zeit in Monat-Jahr formatieren
df['Zeit'] = df['InbetriebnahmeDatum'].apply(convert_json_date).dt.strftime('%m-%Y')

#Speicher in Datenablage
df.to_csv('/content/drive/MyDrive/Datenablage/et_wind_n_raw.csv', index=False)

#Nur Onshore-Windräder
df = df.loc[df['LageEinheit'] == 888]

# Neue Spalte erstellen und durch 1000 teilen
df['Bruttoleistung in kW'] = df['Bruttoleistung'] / 1000

#Lösche überflüssige Spalten
df.drop(columns=['InbetriebnahmeDatum', 'LageEinheit', 'LageEinheitBezeichnung', 'Zeit','Bruttoleistung'], inplace=True)

#Umbenennen
df = df.rename(columns={'Bruttoleistung':'Bruttoleistung in kW'})

# Speichern der Datentabelle
df.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_wind_n_d.csv', index=False)

#Anzahl der Windräder
anzahl_wr = df.shape[0]
anzahl_wr

jetzt = datetime.now(pytz.timezone('Europe/Berlin')).strftime('%d.%m.%Y')

CHART_ID = "lQmbH"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)
dw.update_description(
    chart_id=CHART_ID,
    intro = 'Seit Beginn des Jahres wurden <b><span style="color: #447ebc;"><strong>{}</strong></span></b> Windkraftanlagen in Betrieb genommen.'.format(str(anzahl_wr)) + '<br><br> <b> Stand: </b> {}'.format(jetzt),
    byline="dena",
    source_name="MaStR",
    source_url="https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/ErweiterteOeffentlicheEinheitenuebersicht?"
)


{'publicId': 'lQmbH',
 'language': 'de-DE',
 'theme': 'dena',
 'id': 'lQmbH',
 'type': 'd3-maps-symbols',
 'title': 'Ausbau der Windenergie in Deutschland',
 'lastEditStep': 5,
 'publishedAt': '2024-10-18T10:00:16.000Z',
 'publicUrl': 'https://datawrapper.dwcdn.net/lQmbH/76/',
 'publicVersion': 76,
 'deleted': False,
 'deletedAt': None,
 'forkable': False,
 'isFork': False,
 'metadata': {'data': {'changes': [],
   'transpose': False,
   'vertical-header': True,
   'horizontal-header': True,
   'auto-geocode': False,
   'column-order': [0, 1, 2, 3],
   'column-format': {'Plz': {'type': 'text'},
    'Breitengrad': {'type': 'number'},
    'Laengengrad': {'type': 'number'},
    'Bruttoleistung': {'type': 'number'},
    'Bruttoleistung in kW': {'type': 'number'}},
   'upload-method': 'copy'},
  'describe': {'source-name': 'MaStR',
   'source-url': 'https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/ErweiterteOeffentlicheEinheitenuebersicht?',
   'intro': 'Seit Beginn des Jahres 

In [ ]:
# NEU Solar-PV Anzahl -Netto- (MaStR)
#Energieträger entspricht Stahlungsenergie
#Inbetriebnahme nach 31.12.2023

def convert_json_date(json_date):
    # Extrahiere die Millisekunden aus dem JSON-Datum
    milliseconds = int(re.search(r'\d+', json_date).group(0))

    # Konvertiere Millisekunden in Sekunden und dann in ein Datumsobjekt
    date = datetime.utcfromtimestamp(milliseconds / 1000)

    return date

# Aktuelles Jahr bekommen und um eins reduzieren
current_year = datetime.now().year - 1

####### Inbetriebnahmen

# Starte mit Seite 1
seite = 1
dfs_1 = []

while True:
    # Link mit der aktuellen Seite füllen
    url_1 = f"https://www.marktstammdatenregister.de/MaStR/Einheit/EinheitJson/GetErweiterteOeffentlicheEinheitStromerzeugung?sort=&page={seite}&pageSize=25000&group=&filter=Energietr%C3%A4ger~eq~%272495%27~and~Inbetriebnahmedatum%20der%20Einheit~gt~%2731.12.{current_year}%27"

    # Anfrage senden
    response = requests.get(url_1)
    data_1 = response.json()

    # Überprüfe, ob Daten zurückgegeben wurden
    if not data_1['Data']:
        break

    data_1 = data_1['Data']

    # Daten in DataFrame laden
    df = pd.DataFrame(data_1, columns=['MaStRNummer','Plz', 'Breitengrad', 'Laengengrad', 'LageEinheit', 'LageEinheitBezeichnung', 'InbetriebnahmeDatum','Bruttoleistung'])

    # Zeit in Monat-Jahr formatieren
    df['Zeit'] = df['InbetriebnahmeDatum'].apply(convert_json_date).dt.strftime('%m-%Y')

    # Postleitzahl zu 2-stellem Code (für DW)
    df['ZIP'] = df['Plz'].str[:2]

    # Einheit Bruttoleistung ändern
    df['Bruttoleistung in MW'] = df['Bruttoleistung']/1000

    dfs_1.append(df)

    # Inkrementiere Seite für die nächste Anfrage
    seite += 1

# Konkateniere alle DataFrames
df_1 = pd.concat(dfs_1, ignore_index=True)

########## Stilllegungen

# Starte mit Seite 1
seite = 1
dfs_2 = []

while True:
    # Link mit der aktuellen Seite füllen
    url_2 = f"https://www.marktstammdatenregister.de/MaStR/Einheit/EinheitJson/GetErweiterteOeffentlicheEinheitStromerzeugung?sort=&page={seite}&pageSize=25000&group=&filter=Energietr%C3%A4ger~eq~%272495%27~and~Datum%20der%20endg%C3%BCltigen%20Stilllegung~gt~%2731.12.{current_year}%27"

    # Anfrage senden
    response = requests.get(url_2)
    data_2 = response.json()

    # Überprüfe, ob Daten zurückgegeben wurden
    if not data_2['Data']:
        break

    data_2 = data_2['Data']

    # Daten in DataFrame laden
    df_ = pd.DataFrame(data_2, columns=['MaStRNummer','Plz', 'Breitengrad', 'Laengengrad', 'LageEinheit', 'LageEinheitBezeichnung', 'EndgueltigeStilllegungDatum','Bruttoleistung'])

        # Zeit in Monat-Jahr formatieren
    df_['Zeit'] = df_['EndgueltigeStilllegungDatum'].apply(convert_json_date).dt.strftime('%m-%Y')

    # Postleitzahl zu 2-stellem Code (für DW)
    df_['ZIP'] = df_['Plz'].str[:2]

    # Einheit Bruttoleistung ändern
    df_['Bruttoleistung in MW'] = df_['Bruttoleistung']/1000

    dfs_2.append(df_)

    # Inkrementiere Seite für die nächste Anfrage
    seite += 1

# Konkateniere alle DataFrames
df_2 = pd.concat(dfs_2, ignore_index=True)

# Ermittlung Nettozubau
summe_df_1 = df_1['Bruttoleistung in MW'].sum()
summe_df_2 = df_2['Bruttoleistung in MW'].sum()

summe_df = summe_df_1 - summe_df_2
summe_df = summe_df.round(1)

# Update Map
df_1 = df_1[df_1['Plz'].notna() &
                     df_1['Breitengrad'].notna() &
                     df_1['Laengengrad'].notna() &
                     df_1['Bruttoleistung'].notna()]
df_1.drop(columns=['MaStRNummer','Plz', 'InbetriebnahmeDatum', 'LageEinheit', 'LageEinheitBezeichnung', 'Zeit', 'Bruttoleistung'], inplace=True)

# Speichern der Datentabelle
df_1.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_solar_n_d.csv', index=False)

jetzt = datetime.now(pytz.timezone('Europe/Berlin')).strftime('%d.%m.%Y')

# Update der Map
CHART_ID = "J4WGR"
dw.add_data(chart_id=CHART_ID, data=df_1)
dw.publish_chart(chart_id=CHART_ID, display=False)
dw.update_description(
    chart_id=CHART_ID,
    intro='Seit Beginn des Jahres wurden <b><span style="color: #f39200;"<strong>{}</strong></span></b> MW Solarkapazität (netto) in Betrieb genommen. Das Ausbauziel 2024 liegt bei 13 GW.'.format(str(summe_df))+ '<br><br> <b> Stand: </b> {}'.format(jetzt),
    byline="dena",
    source_name="MaStR",
    source_url="https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/ErweiterteOeffentlicheEinheitenuebersicht?")

<ipython-input-18-4742f6dd4e80>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.drop(columns=['MaStRNummer','Plz', 'InbetriebnahmeDatum', 'LageEinheit', 'LageEinheitBezeichnung', 'Zeit', 'Bruttoleistung'], inplace=True)


{'publicId': 'J4WGR',
 'language': 'de-DE',
 'theme': 'dena',
 'id': 'J4WGR',
 'type': 'd3-maps-symbols',
 'title': 'Ausbau der Solarenergie in Deutschland',
 'lastEditStep': 5,
 'publishedAt': '2024-10-18T10:24:12.000Z',
 'publicUrl': 'https://datawrapper.dwcdn.net/J4WGR/70/',
 'publicVersion': 70,
 'deleted': False,
 'deletedAt': None,
 'forkable': False,
 'isFork': False,
 'metadata': {'data': {'changes': [],
   'transpose': False,
   'vertical-header': True,
   'horizontal-header': True,
   'auto-geocode': False,
   'column-order': [0, 1, 2],
   'column-format': {'Breitengrad': {'type': 'number'},
    'Laengengrad': {'type': 'number'}},
   'upload-method': 'copy'},
  'describe': {'source-name': 'MaStR',
   'source-url': 'https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/ErweiterteOeffentlicheEinheitenuebersicht?',
   'intro': 'Seit Beginn des Jahres wurden <b><span style="color: #f39200;"<strong>11961.8</strong></span></b> MW Solarkapazität (netto) in Betrieb genommen.

In [ ]:
# Update Installierte Leistung (Fraunhofer ISE)

# JSON-Daten von der URL abrufen
url = "https://www.energy-charts.info/charts/installed_power/data/de/p_inst_year.json"
response = requests.get(url)
data = response.json()

# Extrahiere die xAxisValues einmalig
xAxisValues = data[0]["xAxisValues"]

# DataFrame erstellen
df = pd.DataFrame()

# Iteriere über jedes Element in data
for item in data:
    # Extrahiere den Namen
    name = item["name"][0]["de"]

    # Extrahiere die data
    data_values = item["data"]

    # Füge die data dem DataFrame hinzu
    df[name] = data_values

# Verwende die xAxisValues als Index und konvertiere ihn in ein datetime-Objekt
df.index = pd.to_datetime(xAxisValues, format='%Y')

# Filtere das aktuelle Jahr aus
current_year = datetime.now().year
df = df[df.index.year < current_year]

# Liste der Spalten mit "speicher" im Namen erstellen
columns_to_drop = [col for col in df.columns if 'speicher' in col.lower()]

# Entferne die entsprechenden Spalten
df.drop(columns=columns_to_drop, inplace=True)

# Benenne die Spalte "Sonstige, nicht erneuerbar" in "Sonstige Konventionelle" um
df.rename(columns={'Sonstige, nicht erneuerbar': 'Sonstige Konventionelle'}, inplace=True)

# Konvertiere den Index in das gewünschte Format '%Y'
df.index = df.index.strftime('%Y')

# Den Index als Spalte hinzufügen und umbenennen
df.reset_index(inplace=True)
df.rename(columns={'index': 'Jahr'}, inplace=True)

# Transponiere den DataFrame
df = df.transpose().reset_index()

# Benenne die erste Spalte um
df.rename(columns={0: 'Quelle'}, inplace=True)

# Setze die nullte Zeile als Header
df.columns = df.iloc[0]
df = df[1:]

# Ausgabe des transponierten DataFrames
print(df)

# Speichern des Datensatzes
df.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_inst_leistung_y.csv', index=False)

# Update des Charts
CHART_ID = "dIIXv"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)

0                      Jahr    2002    2003    2004    2005    2006    2007  \
1               Kernenergie  23.538  22.042  21.443  21.343  21.144  21.244   
2                Braunkohle  20.304  20.868  20.774   20.68  20.492   21.15   
3                Steinkohle  28.294   28.67  30.362  27.636  26.978  27.542   
4                    Erdgas    20.3    19.5    19.4    20.6    21.2    21.3   
5                 Mineralöl     5.3     5.1     5.6     5.5     5.5     5.4   
6   Sonstige Konventionelle     NaN     NaN     NaN     NaN     NaN     NaN   
7                Laufwasser   4.937   4.953   5.186    5.21   5.193   5.137   
8                  Biomasse   0.996    1.12   1.323   1.879   2.159   2.939   
9             Wind offshore     0.0     0.0     0.0     0.0     0.0     0.0   
10             Wind onshore  11.976  14.381  16.419  18.248  20.474  22.116   
11                    Solar   0.296   0.435   1.105   2.056   2.899    4.17   

0     2008    2009    2010  ...    2014    2015    

{'data': {'publicId': 'dIIXv',
  'language': 'de-DE',
  'theme': 'dena',
  'id': 'dIIXv',
  'type': 'stacked-column-chart',
  'title': 'Installierte Netto-Leistung zur Stromerzeugung in Deutschland',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T10:24:17.940Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/dIIXv/33/',
  'publicVersion': 33,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'column-format': {},
    'upload-method': 'copy'},
   'describe': {'source-name': 'Fraunhofer ISE',
    'source-url': 'https://www.energy-charts.info/charts/installed_power/chart.htm?l=de&c=DE&year=-1&expansion=installed_power',
    'intro': 'in GW',
    'byline': 'dena',
    'aria-description': '',
    'number-format': '-',
    'number-divisor': 0,
    'number-append': '',
    'number-prepend': '',
    'hide-title': False},
   'visualize': 

In [ ]:
# UPDATE Ausschreibungen Wind EEG (BNetzA)

# URL zur Excel-Dateien
url = "https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/Ausschreibungen/Statistiken/Statistik_Onshore.xlsx?__blob=publicationFile&v=5"

# Herunterladen der Excel-Dateien
response = requests.get(url)
excel_data = BytesIO(response.content)

# Lese die Excel-Datei ein
df = pd.read_excel(excel_data, header=None)

# Initialisiere eine leere Liste für den kombinierten Header
combined_header = []

# Iteriere über die Spalten und kombiniere die Header-Zellen aus Zeile 7 und 8
prev_header = None
for col in range(len(df.columns)):
    header_cell_7 = df.iloc[6, col]
    header_cell_8 = df.iloc[7, col]

    # Überprüfe, ob Zelle in Zeile 7 leer ist, und verwende vorherigen Wert
    if pd.isna(header_cell_7):
        combined_header.append(f"{prev_header} - {header_cell_8}")
    else:
        combined_header.append(f"{header_cell_7} - {header_cell_8}" if pd.notna(header_cell_8) else header_cell_7)
        prev_header = header_cell_7

# Setze den kombinierten Header als neuen Header für den DataFrame
df.columns = combined_header

# Entferne alle Zeilen bis einschließlich Zeile 7 aus dem DataFrame
df = df.iloc[8:].reset_index(drop=True)

# Beschränke den DataFrame so, dass er in der Zeile endet, bevor "Verfahren" den Wert "Summe" erreicht
end_index = df[df['Verfahren'] == 'Summe'].index[0] if 'Verfahren' in df.columns else len(df)
df = df.iloc[:end_index]

# Wähle nur die gewünschten Spalten aus
selected_columns = ['Gebotstermin', 'Ausschreibungsvolumen (kW)', 'Zuschlagsmenge (kW)']
df = df[selected_columns]

# Beschränke den DataFrame auf die ersten drei Spalten
df = df.iloc[:, :3]

# Umrechne kW in MW für die Spalten "Gebotsmenge (kW)" und "Zuschlagsmenge (kW)"
df['Ausschreibungsvolumen'] = df['Ausschreibungsvolumen (kW)'] / 1000000
df['Zuschlagsmenge'] = df['Zuschlagsmenge (kW)'] / 1000000

# Entferne die ursprünglichen Spalten "Gebotsmenge (kW)" und "Zuschlagsmenge (kW)"
df = df.drop(['Ausschreibungsvolumen (kW)', 'Zuschlagsmenge (kW)'], axis=1)

# Ändere das Datumsformat der Spalte "Gebotstermin" in "dd-MM-YYYY"
df['Gebotstermin'] = pd.to_datetime(df['Gebotstermin']).dt.strftime('%d.%m.%Y')

# Transponiere den DataFrame
df = df.transpose()

# Setze die 0. Zeile als neuen Header
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

# Setze den Index zurück
df = df.reset_index()

# Speichern des Datensatzes
df.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_ausschreibungen_wind.csv', index=False)

# Update des Charts
CHART_ID = "0tvrU"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)

{'data': {'publicId': '0tvrU',
  'language': 'de-DE',
  'theme': 'dena',
  'id': '0tvrU',
  'type': 'grouped-column-chart',
  'title': 'Ausschreibung von Windenergieanlagen an Land nach dem Erneuerbaren-Energien-Gesetz (EEG)',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T10:24:23.833Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/0tvrU/29/',
  'publicVersion': 29,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'upload-method': 'copy'},
   'describe': {'source-name': 'BNetzA',
    'source-url': 'https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/Ausschreibungen/Wind_Onshore/BeendeteAusschreibungen/start.html',
    'intro': 'in GW',
    'byline': 'dena',
    'aria-description': '',
    'number-format': '-',
    'number-divisor': 0,
    'number-append': '',
    'number-prepend': '',
    'hide-title': False},

In [ ]:
# Produktion von PV, Solarthermal und Wärmepumpen (Destatis)

# URL zur CSV-Datei
url = "https://www.dashboard-deutschland.de/api/attachments/get?tileid=tile_1663667563512&attachmentName=Produktion%20von%20Solarkollektoren%2C%20Solarmodulen%20und%20W%C3%A4rmepumpen.csv"

# Herunterladen der CSV-Dateien
response = requests.get(url)
csv_data = BytesIO(response.content)

# Einlesen der CSV
df = pd.read_csv(csv_data, encoding='utf-8', delimiter=';', header=5)

# Abspeichern der Datentabelle
df.to_csv('/content/drive/MyDrive/Fertige_Tabellen/et_prod_sol_the_wp_m.csv', index=False)


In [ ]:
# NEU Endenergieeffizienz vs BIP und Endenergieverbrauch

# Daten Endenergieverbrauch
data_filter = {'nrg_bal':['FEC2020-2030'],'geo':['DE'],'unit': ['TJ'], 'siec':['TOTAL']}
df = eurostat.get_data_df('nrg_bal_s', filter_pars=data_filter)

df = df.drop(columns=['freq','nrg_bal','siec','unit'])

df.loc[0, 'geo\TIME_PERIOD'] = 'EEnergieV_DE'

# Einheit von TJ zu PJ ab Spalte 1990
columns_to_divide = df.loc[:, '1990':]
df.loc[:, '1990':] = columns_to_divide / 1000

# Daten GDP
# Dateneinlese
data_filter = {'geo':['DE'],'unit': ['CLV15_MEUR'], 'na_item':['B1GQ']}
df_gdp = eurostat.get_data_df('nama_10_gdp', filter_pars=data_filter)

df_gdp = df_gdp.drop(columns=['freq','unit','na_item'])

columns_to_drop = df_gdp.loc[:, '1975':'1989'].columns
df_gdp = df_gdp.drop(columns=columns_to_drop)

df_gdp.loc[0, 'geo\TIME_PERIOD'] = 'BIP_DE'

# Daten für Energieeffizienz
df_all = pd.concat([df, df_gdp], ignore_index=True)

# Transponieren
df_all = df_all.transpose()
df_all.reset_index(inplace=True)
df_all.columns = df_all.iloc[0]
df_all = df_all[1:].reset_index(drop=True)

# Berechnung Endenergieeffizienz
df_all['Deutschland_EE'] = df_all['BIP_DE'] / df_all['EEnergieV_DE']

# Speichern der Rohdaten
df_all.to_csv('/content/drive/MyDrive/Datenablage/ww_energieverbrauch_raw.csv', index=False)


# Index 1990 mit BIP und Energieverbrauch
df_en_wa = df_all[['geo\TIME_PERIOD', 'Deutschland_EE','EEnergieV_DE','BIP_DE']]

# Wachstum berechnen
basiswert = df_en_wa.loc[1, 'Deutschland_EE']
df_en_wa['Endenergieeffizienz'] = (df_en_wa['Deutschland_EE'] - basiswert) / basiswert * 100
df_en_wa['Endenergieeffizienz'] += 100

basiswert = df_en_wa.loc[1, 'EEnergieV_DE']
df_en_wa['Endenergieverbrauch'] = (df_en_wa['EEnergieV_DE'] - basiswert) / basiswert * 100
df_en_wa['Endenergieverbrauch'] += 100

basiswert = df_en_wa.loc[1, 'BIP_DE']
df_en_wa['BIP'] = (df_en_wa['BIP_DE'] - basiswert) / basiswert * 100
df_en_wa['BIP'] += 100

df_en_wa = df_en_wa[['geo\TIME_PERIOD', 'Endenergieeffizienz','Endenergieverbrauch','BIP']]

df_en_wa.to_csv('/content/drive/MyDrive/Fertige_Tabellen/ww_endegvbr_idx_j.csv', index=False)

# Update des Charts
CHART_ID = "1FQ9o"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)


<ipython-input-22-52ed71b68da0>:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df, df_gdp], ignore_index=True)
<ipython-input-22-52ed71b68da0>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_wa['Endenergieeffizienz'] = (df_en_wa['Deutschland_EE'] - basiswert) / basiswert * 100
<ipython-input-22-52ed71b68da0>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

{'data': {'publicId': '1FQ9o',
  'language': 'de-DE',
  'theme': 'dena',
  'id': '1FQ9o',
  'type': 'd3-lines',
  'title': 'Endenergieeffizienz',
  'lastEditStep': 5,
  'publishedAt': '2024-10-18T10:24:30.469Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/1FQ9o/21/',
  'publicVersion': 21,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': True,
    'vertical-header': True,
    'horizontal-header': True,
    'upload-method': 'copy'},
   'describe': {'source-name': 'Eurostat',
    'source-url': 'https://ec.europa.eu/eurostat/de/data/database',
    'intro': '1991 = 100',
    'byline': 'dena',
    'aria-description': '',
    'number-format': '-',
    'number-divisor': 0,
    'number-append': '',
    'number-prepend': '',
    'hide-title': False},
   'visualize': {'dark-mode-invert': True,
    'highlighted-series': [],
    'highlighted-values': [],
    'sharing': {'enabled': False,
     'url': 'https://